# 3 找到最大的时间消费者

## 3.1 一个时钟周期有多长?

在本手册中，我使用CPU时钟周期而不是秒或微秒作为时间度量。
这是因为电脑的速度非常不同。如果我写出今天需要10微秒的时间，那么下一代计算机可能只需要5微秒，我的手册很快就会过时。
但是如果我写出某些东西需要10个时钟周期，那么即使CPU时钟频率加倍，仍然需要10个时钟周期。

时钟周期的长度是时钟频率的倒数。例如，如果时钟频率为2 GHz，则时钟周期的长度为
$$ \frac{1}{2GHz} = 0.5ns $$

一台计算机上的时钟周期并不总是与另一台计算机上的时钟周期相当。
奔腾4（NetBurst）CPU被设计为比其他CPU更高的时钟频率，但是与其他CPU相比，它需要使用更多的时钟周期来执行同一段代码。

假设程序中的循环重复1000次，并且循环内有100个浮点运算（加法，乘法等）。
如果每个浮点运算需要5个时钟周期，那么我们可以大致估计在2 GHz CPU上该环路需要 `1000 * 100 * 5 * 0.5 ns = 250μs`。
我们应该尝试优化这个循环吗？ 当然不！ 250μs小于刷新屏幕所需时间的1/50。用户感觉不到任何延迟。
但是如果这个循环包含在另一个也重复1000次的循环内，那么我们估计的计算时间为250毫秒。
这种延迟刚刚够长到引起人们的注意，但还不至于长到让人厌烦。
我们可能会决定做一些测量，看看我们的估算是否正确，或者估算的时间是否超过250毫秒。
如果响应时间长到用户实际上需要等待结果，那么我们会考虑是否有可以改进的地方。

## 3.2 使用分析器找到热点

在开始优化任何事情之前，您必须确定程序的关键部分。
- 在某些程序中，超过99％的时间花在最内层循环中进行数学计算。
- 在其他程序中，99％的时间用于读取和写入数据文件，而不到1％用于实际处理这些数据。

优化关键代码的非常重要，而不是优化仅占用一小部分时间的代码。
优化不太关键的代码部分不仅浪费时间，还会使代码更混乱，更难调试和维护。

大多数编译器软件包都包含一个分析器，它可以告诉每个函数被调用了多少次，占用了多少时间。
还有些第三方分析器，如AQtime，英特尔VTune和AMD CodeAnalyst。

有几种不同的性能分析方法：

- 插桩（Instrumentation）。编译器在每次函数调用时插入额外的代码来计算函数被调用的次数以及需要多少时间。
- 调试。 分析器在每个函数或每个代码行处，插入临时调试断点。
- 基于时间的采样：分析器告诉操作系统产生中断，例如，每毫秒一次。分析器计算程序每个部分中断发生的次数。这不需要修改正在测试的程序，但不太可靠。
- 基于事件的采样：分析器告诉CPU在某些事件时产生中断，例如，每当一千次高速缓存未命中时。这样就可以发现程序的哪一部分具有最多的高速缓存未命中，分支错误预测，浮点异常等。基于事件的采样需要CPU特定的分析器。对于Intel CPU，需要使用Intel VTune，AMD CPU则使用AMD CodeAnalyst。

不幸的是，性能分析器（profilers）往往不可靠。 由于技术问题，他们有时会产生令人误解的结果，或完全失败。

分析器的一些常见问题是：
- 粗粒度时间测量。如果是以毫秒分辨率来测量，但关键函数仅仅需要几微秒执行，则测量结果可能变得不精确或者就为零。

- 执行时间太短或太长。如果被测试程序在短时间内完成，那么采样产生的分析数据太少。如果程序执行时间太长，那么分析器可能会采集到太多的数据，超过它的处理能力。

- 等待用户输入。许多程序花费大部分时间等待用户输入或等待网络资源。这些等待的时间也会被分析器捕捉到。为了使性能分析可行，可能有必要修改程序，以使用一组测试数据而不是从用户输入。

- 来自其他进程的干扰。分析器不仅测量被测试程序的时间，还测量在同一台计算机上运行的所有其他进程（包括分析器本身）所用的时间。

- 函数地址在优化程序中被隐藏。分析器通过地址，识别程序中的所有性能热点，并尝试将这些地址转换为函数名。但是高度优化的程序经常重新组织生成的代码，使得函数名和代码地址之间没有明确的对应关系。内联函数的名称可能对分析器根本就不可见。结果将会生成误导性的报告。

- 使用代码的调试版本。 某些分析器要求您正在测试的代码包含调试信息以便识别单个函数或代码行。代码的调试版本是未被优化的。

- 在CPU不同的核之间跳转。在多核CPU上处理器上，进程或线程不一定会保持在相同的处理器内核上运行，但事件计数器却可以。这会导致在多个CPU内核之间跳转的线程的无意义事件计数。 您可能需要通过设置线程亲和性掩码，来将线程锁定到特定的CPU内核。

- 可重复性差。程序执行的延迟可能由不可重现的随机事件引起。任务切换和垃圾回收等事件可随机发生，并使程序的某些部分看起来比平时花费更长的时间。

使用性能分析器有多种方式。
一个简单的方法是在调试器中运行程序，并在程序运行时按下中断。
如果有一个热点使用90％的CPU时间，那么有90％的机会在这个热点地区发生中断。重复中断几次可能足以确定一个热点。
在调试器中查看调用堆栈来确定热点相关的情况。

有时，识别性能瓶颈的最好方法是将测量插桩代码直接放入代码中，而不是使用现成的分析器。
这并不能解决与分析相关的所有问题，但它通常会提供更可靠的结果。
如果您对分析器的工作方式不满意，那么您可以将所需的测量插桩代码放入程序本身。
您可以添加计数器变量来计算程序的每个部分执行的次数。
此外，您还可以读取每个最重要或最关键部分的前后时间点，以测量每个部分需要花费多少时间。
有关此方法的进一步讨论，请参阅第157页。

您的测量代码应该被`#if`指令包含住，以便在代码的最终版本中禁用它。
在代码自身中插入自己的分析插桩代码，是在程序开发过程中跟踪性能的非常有用的方法。

如果时间间隔很短，时间测量可能需要非常高的分辨率。
在Windows中，您可以使用`GetTickCount`或`QueryPerformanceCounter`函数获取毫秒分辨率。
使用CPU中的时间戳记计数器可以获得更高的分辨率，该计数器以CPU时钟频率计数（在Windows中：`__rdtsc()`）。

如果线程在不同的CPU内核之间跳转，则时间戳计数器将变为无效。
您可能需要在时间测量过程中将线固定到特定的CPU内核以避免这种情况。（在Windows中，`SetThreadAffinityMask`，在Linux中，`sched_setaffinity`）。

程序应该用一组真实的测试数据进行测试。
测试数据应包含典型的随机性，以获得缓存未命中和分支预测失误的实际次数。

当找到程序中最耗时的部分时，重要的是只将优化工作集中在耗时的部分上。
关键代码段可以通过第157页所述的方法进一步测试和深入探查。

分析器对于查找与CPU密集型代码相关的问题非常有用。
但是许多程序使用更多时间加载文件或访问数据库，网络和其他资源，而不是进行算术操作。
以下部分讨论最常见的时间消耗情形。


## 3.3 程序安装

安装程序包花费的时间通常不被看做软件优化问题。但它肯定是可以窃取用户时间的。
如果软件优化的目标是为用户节省时间，那么安装软件包并使其工作所花费的时间不能忽视。
由于现代软件的高度复杂性，安装过程花费一个多小时并不罕见。为了查找和解决兼容性问题，用户不得不多次重新安装软件包，这也常常发生。

在决定是否要将软件包搭建在需要大量文件的复杂框架上时，软件开发人员应该考虑安装时间和兼容性问题。

安装过程应始终使用标准化的安装工具。
应该允许用户在安装启动时选择安装选项，以便剩下的安装过程可以无人照管直到结束。
卸载也应该以标准化的方式进行。

## 3.4 自动更新

许多软件程序会定期通过Internet自动下载更新。
- 某些程序每次启动计算机时都会搜索更新，即使程序从未使用过。安装了许多此类程序的计算机可能需要几分钟才能启动，这完全是浪费用户的时间。
- 其他程序，每次程序启动时，会占用时间搜索更新。如果当前版本满足用户需求，用户可能不需要更新。

搜索更新应该是可选的，默认情况下关闭，除非有非常重要的安全更新。
更新过程应该在低优先级的线程中运行，并且仅在程序实际使用时才运行。
程序没在使用时，不应该让后台进程运行。
对于下载好的程序更新的安装，应该推迟到程序关闭并重新启动。

操作系统的更新可能特别耗时。有时需要花费数小时才能把自动更新安装到操作系统。
这是非常有问题的，因为这些耗时的更新不知道什么时候，就可能在不方便的时候出现。
如果用户在离开工作场所之前必须关闭或注销计算机出于安全原因，系统禁止用户在更新过程中关闭计算机，则这可能是一个非常大的问题。

如果用户在离开工作场所之前，出于安全原因必须关闭或注销计算机，但是系统禁止用户在更新过程中关闭计算机，则这可能是一个非常大的问题。

## 3.5 程序加载

加载程序经常比执行程序花费更多的时间。
对于基于大型运行时框架，中间代码，解释器，即时（JIT）编译器等的程序，加载时间可能会长得很烦人。例如用Java，C#，Visual Basic等编写的程序通常就是这种情况。

但是，即使通过C++编译实现的程序，程序加载也可能是耗时的。
这种情况通常发生在如果程序使用：
- 很多运行时DLL（动态链接库或共享对象）
- 资源文件
- 配置文件
- 帮助文件和数据库

程序启动时，操作系统可能不去加载大型程序的所有模块。
某些模块只有在需要时才去加载。如果内存大小不足，它们可能会被交换到硬盘。

用户期望即时响应按键或鼠标移动等简单操作。
如果因为需要从磁盘加载模块或资源文件，这种响应延迟了几秒钟，用户是不会接受的。
吃内存的内存饥饿型应用程序迫使操作系统将内存交换到磁盘。
内存交换是对鼠标移动或按键等简单事情的响应时间过长的常见原因。

应该避免在硬盘上散布过多数量的DLL，配置文件，资源文件，帮助文件等。
少量一些文件，最好在与.exe文件相同的目录下，是可以接受的。

## 3.6 动态链接和位置无关代码

函数库可以作为静态链接库（.lib，.a）或动态链接库（也称为共享对象（.dll，.so））来实现。
有几个因素可以使动态链接库比静态链接库慢。这些因素在下面的第149页详细解释。

位置无关代码用于类Unix系统中的共享目标文件。
默认情况下，Mac系统经常使用与位置无关的代码。
与位置无关的代码效率低下，尤其是在32位模式下，原因如下第149页所述。

## 3.7 文件访问

在硬盘上读写文件通常比处理文件中的数据花费更多的时间，特别是在安装了病毒扫描程序，并配置成扫描所有的文件访问。

顺序前向访问文件比随机访问快。
读取或写入大数据块比一次读取或写入一点点要快。
不要一次读取或写入少于几千字节的数据。

您可以将整个文件镜像到内存缓冲区中，并在一次操作中进行读取或写入，而不是以非顺序的方式一点点地读取或写入。

访问最近访问的文件通常比第一次访问文件要快得多。这是因为该文件已被复制到磁盘缓存。

远程或可移动介质（如软盘和USB记忆棒）上的文件可能无法缓存。
这可能会产生相当大的后果。
我曾经创建了一个Windows程序，通过调用`WritePrivateProfileString`创建一个文件，每写入一行时，该文件会被打开和关闭。
由于磁盘缓存，这在硬盘上工作得非常快，但花费了几分钟才能将文件写入软盘。

对于包含数值数据的大文件，以二进制形式存储，比数据以ASCII形式存储时更为紧凑和高效。
二进制数据存储的一个缺点是它不具有人类可读性，不容易移植到具有大端（big-endian）存储的系统。

对于具有许多文件输入/输出操作的程序，优化文件访问比在优化CPU使用更重要。
如果在等待磁盘操作完成时处理器可以执行其他工作，则将文件访问放入单独的线程中可能是有利的。

## 3.8 系统数据库

在Windows中访问系统数据库可能需要几秒钟的时间。
将特定于应用程序的信息存储在单独的文件中比在Windows系统中的大型注册数据库中更有效。
请注意，如果您正在使用诸如`GetPrivateProfileString`和`WritePrivateProfileString`等函数来读写配置文件（* .ini文件），系统可能会将信息存储在数据库中。

## 3.9 其它数据库

许多软件应用程序使用数据库来存储用户数据。
数据库可能会消耗大量CPU时间，RAM和磁盘空间。
在简单情况下，可以用普通的旧式数据文件替换数据库。
数据库查询通常可以通过使用索引，使用集合而不是循环等来优化。
优化数据库查询超出了本手册的范围，但您应该意识到通过优化数据库访问常常有很多事情要做。

## 3.10 图形

图形用户界面会占用大量的计算资源。
通常，需要使用一个特定的图形框架。操作系统可以在其API中提供这样的框架。
在某些情况下，操作系统API和应用程序软件之间还有一层额外的第三方图形框架。
这样的额外框架可能会消耗大量额外的资源。

应用软件中的每个图形操作都是作为对图形库或API函数的函数调用实现的。这些图形库函数或AP函数再调用设备驱动程序。
对图形函数的调用非常耗时，因为它可能会经历多层调用，并且需要切换到保护模式并再次返回。
显然，相比于通过多个函数调用分别绘制每个像素或线段，调用单个图形函数来绘制整个多边形或位图更为高效。

计算机游戏和动画中的，图形对象的计算也很耗时，尤其是在没有图形处理单元的情况下。

各种图形函数库和驱动程序在性能上差别很大。我无法提供具体的推荐。

## 3.11 其它系统资源

对打印机或其他设备的写操作，应该最好以大块数据完成，而不是一次一小块，因为每次调用驱动程序都会包含切换到保护模式并再次返回的开销。

访问系统设备，使用操作系统的高级设施，可能很耗时，因为它可能涉及加载多个驱动程序，配置文件和系统模块。

## 3.12 网络访问

某些应用程序使用互联网或内联网来进行
- 自动更新
- 远程帮助文件
- 数据库访问等。

这里的问题是访问时间无法控制。
- 在简单的测试设置中网络访问可能很快
- 但在网络过载或用户远离服务器的使用情况下，网络访问速度较慢或完全不通。

在决定是否在本地或远程存储帮助文件和其他资源时，应将这些问题考虑在内。
如果需要频繁的更新，则最好在本地镜像远程数据。

访问远程数据库通常需要使用密码登录。
对于许多辛勤工作的软件用户而言，登录过程消耗时间，令人厌烦。
在某些情况下，如果网络或数据库负载过重，登录过程可能需要超过一分钟。

## 3.13 内存访问

与对数据进行计算所需的时间相比，从RAM内存访问数据可能需要相当长的时间。
这就是所有现代计算机都有内存缓存的原因。
通常情况下，CPU有
- 8 - 64千字节的一级数据高速缓存
- 256千字节到2兆字节的2级高速缓存
- 也可能有一个三级缓存

下列情况下，很可能内存访问是程序中最大的时间消耗：
- 如果程序中所有数据的组合大小大于二级缓存，
- 并且数据分散在内存中或以非顺序方式访问

如果它被高速缓存，读取或写入存储器中的变量只需要2-3个时钟周期，但如果没有高速缓存，则需要几百个时钟周期。
有关数据存储，见第26页。有关内存高速缓存，见第89页。

## 3.14 上下文切换

上下文切换是指，
- 在多任务环境中任务的任务的切换
- 在多线程程序中，不同线程之间的切换
- 或大型程序中，不同部分之间的切换

频繁的上下文切换会降低性能，这是因为数据缓存，代码缓存，分支目标缓冲区，分支模式历史，等内容可能不得不被重刷新。

如果分配给每个任务或线程的时间片较小，则上下文切换更频繁。
时间片的长度由操作系统决定，而不是由应用程序决定。

具有多个CPU或具有多个核心的CPU的计算机中，上下文切换的数量较少。

## 3.15 依赖关系链

现代微处理器可以执行乱序执行。
这意味着如果一个软件指定了A然后B的计算，并且A的计算速度很慢，那么微处理器可以在计算A完成之前开始B的计算。
显然，这只有在计算B时不需要A的值时才可能。

为了利用乱序执行，您必须避免长依赖关系链。
依赖关系链是一系列的计算，其中每个计算取决于前一个的结果。
依赖关系链会阻止CPU同时进行多次计算，且阻止乱序执行。
有关如何中断依赖关系链的示例，请参见第105页。

## 3.16 执行单元吞吐量

**时延和执行单元的吞吐量之间有着重要的区别**。
例如，在现代CPU上执行浮点加法可能需要3到5个时钟周期。但是有可能在每个时钟周期开始一个新的浮点加法。
这意味着
- 如果每次加法都取决于前面加法的结果，那么每三个时钟周期只有一次加法。
- 但是如果所有的加法都是独立的，那么你可以在每个时钟周期进行一次加法。

计算密集型程序中，要获得的最高性能，需要满足：
- 在上述章节中提到的各种耗时的情形，没有支配地位
- 并且没有长依赖关系链

在这种情况下，性能仅受执行单元吞吐量的限制，而不受时延或内存访问的限制。

现代微处理器的执行核心切分为几个执行单元。通常，
- 有两个或更多个整数单元，
- 一个或两个浮点加法单元
- 以及一个或两个浮点乘法单元。

这意味着可以在同一时间进行整数加法，浮点加法和浮点乘法运算。

因此，**如果一段代码进行浮点计算，它最好具有平衡的加法和乘法混合一起**。
减法使用与加法相同的执行单位。
除法需要更长的时间。
在浮点运算之间，可以进行整数运算而不降低性能，这是因为整数运算使用不同的执行单元。
例如，执行浮点计算的循环，通常会伴有使用整数运算（递增循环计数器），比较运算（循环计数器与其设定值比较）等。
在大多数情况下，您可以假设这些整数运算不会增加总计算时间。